# W7L1 IP Logical Constraints - In Class Fixed Cost Planes

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Model Definition](#3)<br>
4. [Model Solution](#4)<br>
5. [Redo Model with new constraints](#5)<br>

## 1. Problem Statement<a id = 1></a>

Here we will solve the problem where we need to determine how many of each plane to produce. Each plane has a setup cost and our goal is to maximize our project. 

##### [Back to Top](#Top)

## 2. Data<a id = 2></a>

The data are the sheet Planes tab in w08-c02-planes.xlsx.

In [ ]:
import pandas as pd
import pyomo.environ as pe
#import matplotlib.pyplot as plt

In [ ]:
raw_data = pd.read_excel('w08-c02-planes.xlsx', sheet_name = 'Planes')
raw_data

First we will capture the coefficients which will the the profit per each plane produced, the setup fixed cost, and the units of each mataerial needed for each plane type.

In [ ]:
DV_indexes = ['Rocket', 'Meteor', 'Streak', 'Comet', 'Jet', 'Biplane']
coef = pd.DataFrame(raw_data.iloc[[9, 10, 2, 3, 4, 5, 6, 7], range(1, 7)])
coef.index = ['profit', 'Fixed Setup', 'Steel', 'Copper', 'Plastic', 'Rubber', 'Glass', 'Paint']
coef.columns = DV_indexes
coef

The next tables will give us our max values for our constraints - we have units capacity for each raw material.

In [ ]:
hours = pd.DataFrame(raw_data.iloc[2:8, 7])
hours.columns = ['units']
hours.index = coef.index[2:]
hours

##### [Back to Top](#Top)

## 3. Model Definition<a id = 3></a>

Now let's solve the model.

In [ ]:
model = pe.ConcreteModel()

### Define Decision Variables

__NOTE:__ for this model you have 2 sets of changing cells that you want the solver to determine - the `x`plane quantities and the `yU` values which are binary utilization variables we will used to create the fixed setup cost linking constraints. Let's make the `x` values `NonNegativeIntegers` since we can't make a fraction of a plane.

In [ ]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeIntegers)
model.y = pe.Var(DV_indexes, domain = pe.Binary)
model.x.pprint()
model.y.pprint()

### Define objective function

Note that this will be the total profit which is calculated from the fixed price and variable setup costs. Make sure you see how these are calculated in the Excel sheet before you try to implement here.

In [ ]:
#obj funct max profit * x - setup cost * y
model.obj = pe.Objective(expr = sum(coef.loc['profit', index]*model.x[index] 
                                    for index in DV_indexes) - 
                         sum(coef.loc['Fixed Setup', index]*model.y[index] for index in DV_indexes),
                         sense = pe.maximize)
model.obj.pprint()

### Define Constraints

In [ ]:
#Units Capacity Constraints
model.Steel = pe.Constraint(expr = sum(coef.loc['Steel', index]*model.x[index]
                                       for index in DV_indexes) <= hours.loc['Steel', 'units'])
model.Copper = pe.Constraint(expr = sum(coef.loc['Copper', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Copper', 'units'])
model.Plastic = pe.Constraint(expr = sum(coef.loc['Plastic', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Plastic', 'units'])
model.Rubber = pe.Constraint(expr = sum(coef.loc['Rubber', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Rubber', 'units'])
model.Glass = pe.Constraint(expr = sum(coef.loc['Glass', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Glass', 'units'])
model.Paint = pe.Constraint(expr = sum(coef.loc['Paint',index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Paint', 'units'])

#Linking Constraints with really large arbitrary max demand of 2000
model.LinkRocket = pe.Constraint(expr = model.x['Rocket'] <= 2000*model.y['Rocket'])
model.LinkMeteor = pe.Constraint(expr = model.x['Meteor'] <= 2000*model.y['Meteor'])
model.LinkStreak = pe.Constraint(expr = model.x['Streak'] <= 2000*model.y['Streak'])
model.LinkComet = pe.Constraint(expr = model.x['Comet'] <= 2000*model.y['Comet'])
model.LinkJet = pe.Constraint(expr = model.x['Jet'] <= 2000*model.y['Jet'])
model.LinkBiplane = pe.Constraint(expr = model.x['Biplane'] <= 2000*model.y['Biplane'])

for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

##### [Back to Top](#Top)

## 4. Model Solution<a id = 4></a>

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

### Optimal Objective Value

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

### Optimal Decision Variables

In [ ]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

##### [Back to Top](#Top)

## 5. Now solve the model with the Logical and Disjunctive Constraints<a id=5></a>

Now we want to add the new constraints:
* Suppose that we must pick the Meteor or Comet in our production.
* Suppose also that Rocket must be > 100 or Biplane must be > 100
    

In [ ]:
#NEW BINARY Rocket > 100 Indicator
model.yRocketConst = pe.Var(domain = pe.Binary)
#NEW BINARY Biplane > 100 Indicator
model.yBiplaneConst = pe.Var(domain = pe.Binary)

#New Logical constraint - pick the Meteor or Comet
model.MetORCom = pe.Constraint(expr = model.y['Meteor']+model.y['Comet'] >= 1)

#New Disjunctive constraint - Rocket must be > 100 or Biplane must be > 100
#Rocket−𝑀*yRocket≤100 and Rocket+𝑀(1−𝑦Rocket)≥100
max_dem = 2000
model.RockConst1 = pe.Constraint(expr = model.x['Rocket']-max_dem*model.yRocketConst <= 100)
model.RockConst2 = pe.Constraint(expr = model.x['Rocket']+max_dem*(1-model.yRocketConst) >=100)
#Biplane−𝑀*yBiplane≤100 and Biplane+𝑀(1−𝑦Biplane)≥100
model.BipConst1 = pe.Constraint(expr = model.x['Biplane']-max_dem*model.yBiplaneConst <= 100)
model.BipConst2 = pe.Constraint(expr = model.x['Biplane']+max_dem*(1-model.yBiplaneConst) >=100)
                                 
#Binary var YRocket Const + Binary var YBiplane Const >= 1
model.RockBipLink = pe.Constraint(expr = model.yRocketConst+model.yBiplaneConst >= 1)
model.pprint()

In [ ]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

### New Optimal Objective Value

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

### New Optimal Decision Variables

In [ ]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

##### [Back to Top](#Top)

In [ ]:
for con in model.component_objects(pe.Constraint):
    print(con.lower, con.slack(), con.upper)